# Lab 07: White Box AI

You want to make the perfect toast. You have hacked a toaster with sensors and toasted over 1000 toasts. How can we predict the optimal toasting time for a given toast?

## Preliminary

Let's first generate and explore the data.

We have collected data from 1,000 toasting attempts.

Our Data Features:

-    ```ToastingTime```: How long the toast was in for (seconds).
-    ```BreadThickness```: The thickness of the slice (mm).
-    ```IsFrozen```: A button on our toaster (0 for No, 1 for Yes).
-    ```AmbientTemp```: The room temperature (°C).

Our Target Variable:

- ```ToastState```: The final result (Under-done, Perfect, or Burnt).


Run the code below to generate a pseudo-random dataset. Do not change the seed!

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)

def generate_toaster_data_interactive(n_samples=100000):
    # Feature Generation
    toasting_time = np.random.randint(60, 500, n_samples)
    bread_thickness = np.random.uniform(5, 30, n_samples)
    is_frozen = np.random.randint(0, 2, n_samples)
    ambient_temp = np.random.normal(20, 6, n_samples)

    # Non-linear ideal_time
    base_ideal_time = (30 + 1.5 * bread_thickness + 0.1 * bread_thickness**2 + 0.005 * bread_thickness**3)
    frozen_penalty = is_frozen * (base_ideal_time * (np.exp(0.04 * bread_thickness) - 1))
    unadjusted_ideal_time = base_ideal_time + frozen_penalty
    temp_efficiency_factor = np.exp(0.025 * (ambient_temp - 20))
    ideal_time = unadjusted_ideal_time / temp_efficiency_factor

    # Gaussian perfect_score
    perfect_window_width = 25 + bread_thickness # Thicker bread is more forgiving
    time_diff_sq = (toasting_time - ideal_time)**2
    perfect_score = np.exp(-time_diff_sq / (2 * perfect_window_width**2))

    # Sigmoid burnt score
    burn_onset_offset = 45  # Burning starts to happen ~45s after the ideal time
    burn_rate_k = 0.1       # Controls how quickly it goes from not-burnt to burnt
    time_past_ideal = toasting_time - ideal_time - burn_onset_offset
    burnt_score = 1 / (1 + np.exp(-burn_rate_k * time_past_ideal))

    # Add small noise to scores c.
    perfect_score += np.random.normal(0, 0.3, n_samples)
    burnt_score += np.random.normal(0, 0.3, n_samples)

    # Default to 'Under-done'
    toast_state = np.zeros(n_samples, dtype=int)

    # Classify as 'Perfect' if perfect_score is highest AND above a minimum threshold.
    is_perfect = (perfect_score > burnt_score) & (perfect_score > 0.6)
    toast_state[is_perfect] = 1

    # Classify as 'Burnt' if burnt_score is highest AND above a minimum threshold.
    is_burnt = (burnt_score > perfect_score) & (burnt_score > 0.5)
    toast_state[is_burnt] = 2

    # All other data points remain as 'Under-done'.
    # This creates non-linear boundaries between all three classes.

    df = pd.DataFrame({
        'ToastingTime': toasting_time,
        'BreadThickness': bread_thickness,
        'IsFrozen': is_frozen,
        'AmbientTemp': ambient_temp,
        'ToastState': toast_state
    })
    return df


df = generate_toaster_data_interactive()

# You can now inspect the new dataframe
print(df.head())

We provide the following utility functions to visualise the distribution of a feature relative to another feature as follows:

```displayfeatures(df)``` Displays a matrix of the features plotted against each other

1. Each plot where x != y plots the the values of each feature against each other.
2. Each plot where x == y represents the distribution of the "toastiness" of the bread based on the considered feature.

```displayfeatures3D(df)``` Displays the 3 main features agains each other in 3D


In [ ]:
from matplotlib import pyplot as plt

def displayfeatures(df):
    features = ['ToastingTime', 'BreadThickness', 'AmbientTemp']
    target_col = 'ToastState'
    class_labels = {0: 'Under-done', 1: 'Perfect', 2: 'Burnt'}
    colors = ['blue', 'green', 'red']

    fig, axes = plt.subplots(len(features), len(features), figsize=(12, 12))
    fig.suptitle('Visualizing Toaster Data Relationships', fontsize=16)

    for i, feature_y in enumerate(features):
        for j, feature_x in enumerate(features):
            ax = axes[i, j]

            # Plot histograms on the diagonal
            if i == j:
                ax.hist([df[df[target_col]==k][feature_x] for k in class_labels.keys()],
                        bins=15, histtype='barstacked', color=colors, label=list(class_labels.values()))
            # Plot scatter plots on the off-diagonal
            else:
                for k in class_labels.keys():
                    ax.scatter(df[df[target_col]==k][feature_x],
                            df[df[target_col]==k][feature_y],
                            s=10,
                            alpha=0.5,
                            color=colors[k],
                            label=class_labels[k])

            # Set axis labels
            if i==len(features)-1:
                ax.set_xlabel(feature_x)
            if j==0:
                ax.set_ylabel(feature_y)


    # Create a single legend for the entire figure
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper right')
    plt.tight_layout(rect=[0, 0, 1, 0.96]) # Adjust layout to make space for suptitle
    plt.show()
displayfeatures(df)

In [ ]:
import plotly.express as px
import pandas as pd


def displayfeatures3D(df):
  fig = px.scatter_3d(df,
                      x='ToastingTime',
                      y='BreadThickness',
                      z='AmbientTemp',
                      color='ToastState',  # This column determines the color of the points
                      symbol='ToastState', # Use different symbols for each class
                      title="3D View of Toaster Data",
                      labels={
                          'ToastingTime': 'Toasting Time (s)',
                          'BreadThickness': 'Bread Thickness (mm)',
                          'AmbientTemp': 'Ambient Temperature (C)',
                          'ToastState': 'Final Toast State'
                      },
                      color_discrete_map={ # Explicitly set colors for clarity
                          'Under-done': 'blue',
                          'Perfect': 'green',
                          'Burnt': 'red'
                      },
                      height=700 # Make the plot a bit taller
                    )

  # Our ToastState variable is numerical. Plotly thus considers it as a scale rather than discrete values.
  # To avoid modifying our dataframe, we have to update the traces.
  label_map = {0: 'Under-done', 1: 'Perfect', 2: 'Burnt'}
  fig.for_each_trace(
    lambda trace: trace.update(name=label_map[int(float(trace.name))])
  )

  color_map = {'Under-done': 'blue','Perfect': 'green', 'Burnt': 'red'}
  fig.for_each_trace(
      lambda trace: trace.update(
          marker_color = color_map[trace.name]
      )
  )


  fig.show()

In [4]:
displayfeatures3D(df)

## Step 1: Explore the data

Check what the data looks like using the two functions defined above, as well as the pandas and matplotlib functions seen during class:

- ```df.head()```
- ```df.describe()```
- boxplots, histograms, scatter plots, etc.

The plotting functions ```displayfeatures(df)``` and ```displayfeatures3D(df)``` do not consider whether the bread is frozen or not. You may want to copy these functions to display the data with respect to the bread frozen feature.



## Step 2: Making a Train-Test split

We need to split the data into two sets:
1. a training set that you will use for training the model
2. a testing set that will be used ONLY for testing. This testing set will never be integrated in the training, meaning that the model will have never seen that data before testing.

The reason for this split is that we want our model to **generalise** to new data rather than memorising the answers. Training with all the data without testing with new data will result in some of the worse **overfitting**.

Typically, we tend to use 70% of the data for training and 30% for testing, uniformly distributed across the data. You can extract manually, or use ```sklearn.model_selection.train_test_split```. This function automatically generates four dataframes:
- ```X_train``` The *training* dataframe, containing the 'ToastingTime', 'BreadThickness', 'AmbientTemp', 'IsFrozen' features.
- ```X_test``` The *testing* dataframe, containing the 'ToastingTime', 'BreadThickness', 'AmbientTemp', 'IsFrozen' features.
- ```y_train``` The *training* labels dataframe, containing the 'ToastState' result.
- ```y_test``` The *testing* labels dataframe, containing the 'ToastState' result. This is the **ground truth** that you will compare your testing data against.

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df[['ToastingTime', 'BreadThickness', 'AmbientTemp', 'IsFrozen']],
    df['ToastState'],
    test_size=0.3,
    random_state=42
)

print("-----------------")
print("X_train")
print("-----------------")
print(X_train.describe())
print("-----------------")
print("X_test")
print("-----------------")
print(X_test.describe())
print("-----------------")
print("y_train")
print("-----------------")
print(y_train.describe())
print("-----------------")
print("y_test")
print("-----------------")
print(y_test.describe())

-----------------
X_train
-----------------
       ToastingTime  BreadThickness   AmbientTemp      IsFrozen
count  70000.000000    70000.000000  70000.000000  70000.000000
mean     279.892643       17.495489     20.007879      0.499257
std      127.513041        7.216896      6.004904      0.500003
min       60.000000        5.000164     -6.836685      0.000000
25%      168.000000       11.195513     15.966610      0.000000
50%      281.000000       17.513681     19.997837      0.000000
75%      391.000000       23.755053     24.074372      1.000000
max      499.000000       29.999801     45.519992      1.000000
-----------------
X_test
-----------------
       ToastingTime  BreadThickness   AmbientTemp      IsFrozen
count  30000.000000    30000.000000  30000.000000  30000.000000
mean     279.085867       17.552684     19.968818      0.498433
std      126.999898        7.220138      6.010260      0.500006
min       60.000000        5.000688     -4.783245      0.000000
25%      169.0000

## Step 3: Simple Classifiers



### Logistic Regression

Let's start with logistic regression. We will use ```sklearn.linear_model.LogisticRegression```. Typical use is as follows:

```python
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression model
log_reg = LogisticRegression()

# Train the model
log_reg.fit(X_train, y_train)

# Predict values for our testing data
predictions = log_reg.predict(X_test)
```

The ```LogisticRegression``` function can be initialized with some interesting arguments:
- ```max_iter``` the maximum number of iterations. Default is 100, but as our data is quite complex, you may have to increase to reach convergence
- ```solver``` the solver used to fit the data. Some solvers may be more efficient for some data than others.

Once we have trained the Logistic Regression and done predictions, we can check the accuracy as follows:

```python
from sklearn.metrics import accuracy_score

# Calculate the accuracy between our prediction and the ground truth of our testing data
accuracy = accuracy_score(y_test, predictions)
```


Documentation is here: https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression and here: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

#### Tasks

1. Train a logistic regressor on your training and testing data.
2. Answer the following questions
    1. What is the accuracy?
    2. Change the max number of iterations. How does the accuracy change?
    3. Change the solver. How does the accuracy change? How fast does it converge (use ```log_reg.n_iter_``` to find the number of iterations before convergence)?

In [6]:
# --- TODO: Train a logistic regressor on your training and testing data. --- #
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression model
log_reg = LogisticRegression(max_iter=50,solver='newton-cholesky')

# Train the model
log_reg.fit(X_train, y_train)

# Predict values for our testing data
predictions = log_reg.predict(X_test)

In [7]:
from sklearn.metrics import accuracy_score

# Calculate the accuracy between our prediction and the ground truth of our testing data
accuracy = accuracy_score(y_test, predictions)
print(accuracy)

0.8067


**TODO:**

Double click the cell and answer the following questions

1. What is the accuracy?

0.79667
----
2. Change the max number of iterations. How does the accuracy change?


----
3. Change the solver. How does the accuracy change? How fast does it converge (use ```log_reg.n_iter_``` to find the number of iterations before convergence)?



### Step 4: Decision Tree Classifier

Let us move to decision tree models. Typical use of `sklearn.tree.DecisionTreeClassifier` is very similar to other classifiers:

```python
from sklearn.tree import DecisionTreeClassifier

# Initialize the Decision Tree model
# We can set a random_state for reproducibility of results
dt_clf = DecisionTreeClassifier(random_state=42)

# Train the model
dt_clf.fit(X_train, y_train)

# Predict values
predictions = dt_clf.predict(X_test)
```

The main advantage of a decision tree is its interpretability. However, a major disadvantage is its tendency to **overfit**. A tree that is too deep will learn the noise in the training data and won't generalize well to new data. A key parameter to control this is `max_depth`.

Documentation is here: https://scikit-learn.org/stable/modules/tree.html and here: https://scikit-learn.org/stable/api/sklearn.tree.html

#### Tasks

1.  Train a default `DecisionTreeClassifier` on your training data and evaluate its accuracy on the testing data. How does it compare to the Logistic Regression model?
2.  Trees can be visualized! Use the code below to plot the rules your tree has learned.
    1.   First, train a new tree but limit its depth by setting `max_depth=3` to make the plot readable.
    2.   You can plot the tree using ```plot_tree``` (https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html). Plot the tree.
    3. Can you interpret the first couple of decisions the model makes?
3.  Investigate overfitting. Train multiple decision trees with `max_depth` varying from 1 to 20. For each depth, calculate and record both the **training accuracy** (predicting on `X_train`) and the **testing accuracy** (predicting on `X_test`). Plot both accuracies on the same graph against `max_depth`.
    1.   At what depth does the model start to significantly overfit (i.e., training accuracy keeps increasing while testing accuracy stagnates or drops)?

In [8]:
# --- TODO: Train a default `DecisionTreeClassifier`, and visualize the tree. --- #

**TODO:**

Double click the cell and answer the following questions

4. write a few words to interpret the first couple of decisions the model makes.


----

In [9]:
# --- TODO: Investigate overfitting: maybe train multiple decision trees, and explore the overfitting issue. --- #

**TODO:**

Double click the cell and answer the following questions

5. At what depth does the model start to significantly overfit (i.e., training accuracy keeps increasing while testing accuracy stagnates or drops)?


----

### Step 5: Support Vector Machines (SVM)

Let us use SVM to try to find splits in data that are not necessarily linear. We will use `sklearn.svm.SVC` (Support Vector Classifier). Typical use is as follows:

```python
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Example usage

# Initialize an SVM model with the 'rbf' kernel, and a 'C' hyperparameter at 1
svm_clf = SVC(kernel='rbf', C=1.0, random_state=42)

# Train the model
svm_clf.fit(X_train, y_train)

# Predict the values
svm_predictions = svm_clf.predict(X_test)

# Get the accuracy with respect to the ground truth
accuracy = accuracy_score(y_test, svm_predictions)
```

Documentation is here: https://scikit-learn.org/stable/modules/svm.html and here: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html


#### Tasks

1.  Train a default `SVC` model and evaluate its accuracy on the test set.
2.  The `kernel` is a key parameter. Train and evaluate `SVC` models with the following kernels: `'linear'`, `'poly'`, `'rbf'`, and `'sigmoid'`.
    1. Which one performs best for our toast data?
    2. Why do you think it is?
3.  The `C` parameter is a regularisation parameter. It controls the trade-off between a large margin and a low error rate on the training data. A smaller `C` creates a wider margin but may misclassify more points, while a larger `C` risks creating a more complex, overfitted boundary.
    1.   Using the best kernel you found in the previous step, experiment with different values for `C` (e.g., 0.1, 1, 10, 100). How does the accuracy change?
4. Another parameter is `gamma`. It dictates the influence of a single training example on the model.  A low gamma value means that each training point affects a wider area, leading to a smoother, more generalized decision boundary. A high gamma value restricts the influence to a smaller, closer range, creating a more complex decision boundary that closely follows the training data, which can lead to overfitting. By default, `gamma` is set to `scale`, which means it scales with the data.
    1. Using the best kernel and `C` you found in the previous step, experiment with different values for `gamma` (e.g., 0.1, 1, 10, 100). How does the accuracy change?


In [10]:
# --- TODO: Train a default SVC and evaluate on test dataset --- #

In [11]:
# --- TODO: Train multiple SVCs with different kernels, and evaluate the results. --- #

In [12]:
# --- TODO: with the best kernel in last step, do more experiments with C parameter --- #

In [13]:
# --- TODO: with the best set-up from previous steps, explore parameter gamma. --- #

In practice, we tend to automate the tuning of SVM's hyperparameter. This process involves testing different hyperparameter values to find the combination that minimises loss and maximizes accuracy on a given dataset.

Here, we are going to use the GridSearch algorithm to find the best combination of kernel, C, and gamma.

Task:
1. Run the following script. What is the result? Does it differ from your manual tuning?

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline


# 2. Create a pipeline to scale data and run SVC
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# 3. Define a parameter grid to search
# Use a logarithmic scale for C and gamma
param_grid = {
    'svm__kernel': ['linear','poly','rbf','sigmoid'],
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': [0.001, 0.01, 0.1, 1, 'scale']
}

# 4. Perform the grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

# 5. Evaluate the best model
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.3f}")
print(f"Test set score with best model: {grid_search.score(X_test, y_test):.3f}")

Fitting 5 folds for each of 80 candidates, totalling 400 fits


**TODO:**

Double click the cell and answer the following questions

6. Does it differ from your manual tuning?


----

### Step 6: Random Forest

Now that we have gone through the single expert models, we can move to ensemble learning with Random Forest.

```python
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
rf_predictions = rf_clf.predict(X_test)
```

Documentation is here: https://scikit-learn.org/stable/modules/ensemble.html#random-forests and here: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

#### Tasks

1.  Train a `RandomForestClassifier` from `sklearn.ensemble` on the data. How does its accuracy compare to the single `DecisionTreeClassifier` and the other models?
2.  We can calculate the **feature importance** to show how much each feature contributed to reducing impurity across all the trees.
    *   After training your model, access the feature importances using `your_model.feature_importances_`.
    *   Plot the importance . Which features are most important for determining the state of the toast? Does this align with your intuition?

Hint: to plot the importances, you can use:

```python
importances = rf_clf.feature_importances_
feature_names = X_train.columns
forest_importances = pd.Series(importances, index=feature_names)
```

In [ ]:
# --- TODO: Train a randowm forest classifier  --- #

**TODO:**

Double click the cell and answer the following questions

7. How does its accuracy compare to the single `DecisionTreeClassifier` and the other models?



----

In [ ]:
# --- TODO: find and visualize feature_importance of your model  --- #

**TODO:**

Double click the cell and answer the following questions

8. Which features are most important for determining the state of the toast? Does this align with your intuition?


----

### Step 7: XGBoost

Finally, let's move to XGBoost. This time, we need to  import it from the xgboost library. Usage is the same as the other libraries.

Documentation is here: https://xgboost.readthedocs.io/en/stable/python/python_intro.html

#### Task:

1.  Train a default `xgboost.XGBClassifier` and evaluate its accuracy. Note that if you get a warning about `use_label_encoder=False`, you can safely ignore it or add the parameter `use_label_encoder=False` to the classifier to suppress it.
2.  **Tuning:** XGBoost has many parameters, but two of the most important are `n_estimators` (the number of trees to build) and `learning_rate` (which shrinks the contribution of each tree). A lower learning rate requires more estimators, but can often lead to a more robust model.
    1.   Try training a model with more estimators, for example `n_estimators=500`.
    2.   Now, try that again but with a lower learning rate, for example `n_estimators=500, learning_rate=0.1`.
    3. You can try some other interesting parameters, such as `max_depth` (the max tree depth), or `subsample` (the amount of data passed to a tree).
    4.   How do these changes affect accuracy and the risk of overfitting?
3.  Like Random Forest, XGBoost can also calculate **feature importance**. This is crucial for understanding what drives the model's predictions.
    1.   After training your best XGBoost model, access the feature importances and plot them using the same method as in the Random Forest section.
    2.   Compare the feature importances from XGBoost with those from Random Forest. Do the models agree on which features are most important?
4. Same as SVM, you can run a GridSearch algorithm to find the optimal parameters. Adapt the script provided in the SVM section to find the optimal learning rate, number of estimators, and any other parameter you can think of.


**Create cells of `python` to fill in your code, and of `markdown` to answer the questions.**


### Step 9: Model Comparison and Conclusion

You have now trained and evaluated several different classification models, from simple linear models to powerful ensembles. Let's summarize the results to determine which is best for predicting the perfect toast.

#### Task

1.  Fill in the table below with the best accuracy you achieved for each model type.
2.  Answer the following questions:
    *   Which model gave the highest accuracy?
    *   Which model do you think is the most *interpretable* (easiest to explain to someone)? Why?
    *   Considering both performance and interpretability, which model would you choose to deploy in your "smart toaster"? Justify your choice.

| Model Type | Best Test Accuracy |
| :--- | :--- |
| Logistic Regression | *your_answer_here* |
| Decision Tree | *your_answer_here* |
| Support Vector Machine (SVC) | *your_answer_here* |
| Random Forest | *your_answer_here* |
| XGBoost | *your_answer_here* |

### Step 10: Fine-tune your best models

1.  Please use whatever tools, libraries or methods you know to fine-tune your models.
2.  Pick one with the best performance.
3.  Write down the trainining and evaluation process about this best model below, with anything that you think is necessary.
4.  We will use another blackbox dataset to test your model. The work will be graded based on computing efficiency (runtime) and accuracy.